In [ ]:

import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras
from tensorflow.keras import layers
    
#Load Data
D = np.loadtxt('./coal_percentages.csv', delimiter=',')
P = np.genfromtxt('./Individual_coal_properties.csv', delimiter=',', filling_values=np.nan)
Coke_properties = np.loadtxt('./coke_properties.csv', delimiter=',')

Option = 1
Process_parameters = []
if Option == 1:
    Process_parameters = np.loadtxt('./Process_parameter_for_Rec_Top_Char.csv', delimiter=',')
elif Option == 2:
    Process_parameters = np.loadtxt('./Process_parameter_for_Rec_Stam_Char.csv', delimiter=',')
elif Option == 3:
    Process_parameters = np.loadtxt('./Process_parameter_for_Non_Rec_Stam_Char.csv', delimiter=',')

get_coal_count = 33


# Compute daily vectors
D_tensor = tf.constant(D, dtype=tf.float32)
P_tensor = tf.constant(P, dtype=tf.float32)

daily_vectors = []
for i in range(D_tensor.shape[0]):
    row_vector = []
    for j in range(P_tensor.shape[1]):
        product_vector = tf.multiply(D_tensor[i], P_tensor[:, j])
        row_vector.append(product_vector)
    daily_vectors.append(tf.stack(row_vector))

daily_vectors_tensor = tf.stack(daily_vectors)


daily_vectors_flattened = daily_vectors_tensor.numpy().reshape(74, -1)
Blended_coal_parameters = np.loadtxt('./blended_coal_data.csv', delimiter=',')

input_train, input_test, target_train, target_test = train_test_split(
    daily_vectors_tensor.numpy(), Blended_coal_parameters, test_size=0.2, random_state=42
)

# Scaling
input_scaler = MinMaxScaler()
output_scaler = MinMaxScaler()

input_train_reshaped = input_train.reshape(input_train.shape[0], -1)
input_test_reshaped = input_test.reshape(input_test.shape[0], -1)

input_train_scaled = input_scaler.fit_transform(input_train_reshaped)
input_test_scaled = input_scaler.transform(input_test_reshaped)
input_train_scaled = input_train_scaled.reshape(-1, 33, 15)
input_test_scaled = input_test_scaled.reshape(-1, 33, 15)


target_train_scaled = output_scaler.fit_transform(target_train)
target_test_scaled = output_scaler.transform(target_test)

input_train_scaled = input_train_scaled.reshape(input_train.shape)
input_test_scaled = input_test_scaled.reshape(input_test.shape)
input_train_scaled = input_train_scaled.reshape(-1, 33, 15)
input_test_scaled = input_test_scaled.reshape(-1, 33, 15)


# Define model
modelq = keras.Sequential([
    layers.Input(shape=(33, 15)),
    layers.Flatten(),
    layers.BatchNormalization(),
    layers.Dense(512, activation='relu'),
    layers.Dense(256, activation='leaky_relu', kernel_initializer='he_normal'),
    layers.LayerNormalization(),

    layers.Dense(256, activation='tanh'),
    layers.Dropout(0.3),
    layers.Dense(256, activation='leaky_relu', kernel_initializer='he_normal'),
    layers.Dropout(0.3),

    layers.Dense(128, activation='relu'),
    layers.BatchNormalization(),
    layers.Dense(128, activation='swish', kernel_initializer='he_normal'),
    layers.LayerNormalization(),

    layers.Dense(64, activation='relu'),
    layers.Dropout(0.2),

    layers.Dense(64, activation='swish', kernel_initializer='he_normal'),
    layers.Dropout(0.25),

    layers.Dense(32, activation='relu'),
    layers.BatchNormalization(),

    layers.Dense(32, activation='swish', kernel_initializer='he_normal'),
    layers.LayerNormalization(),
    layers.Dense(15, activation='linear')
])

modelq.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
              loss='mse',
              metrics=['mae'])
modelq.summary()

modelq.fit(input_train_scaled, target_train_scaled, epochs=50, batch_size=8,
               validation_data=(input_test_scaled, target_test_scaled), verbose=0)

if Option == 3:
    Process_parameters = np.pad(Process_parameters, ((0, 0), (0, 2)), mode='constant', constant_values=0)

Conv_matrix = Blended_coal_parameters + Process_parameters

X_train, X_test, y_train, y_test = train_test_split(Conv_matrix, Coke_properties, test_size=0.2, random_state=42)

# Scaling second phase

input__scaler = MinMaxScaler()
output__scaler = MinMaxScaler()
input_train_reshaped = X_train.reshape(X_train.shape[0], -1)
input_test_reshaped = X_test.reshape(X_test.shape[0], -1)

input_train_scaled = input__scaler.fit_transform(input_train_reshaped)
input_test_scaled = input__scaler.transform(input_test_reshaped)

target_train_scaled = output__scaler.fit_transform(y_train)
target_test_scaled = output__scaler.transform(y_test)

 # Build and train second model
rf_model = keras.Sequential([
    layers.Input(shape=(15, 1)),
    layers.Flatten(),
    layers.BatchNormalization(),
    layers.Dense(512, activation='relu'),
    layers.Dense(256, activation='leaky_relu', kernel_initializer='he_normal'),
    layers.LayerNormalization(),
    layers.Dense(256, activation='tanh'),
    layers.Dropout(0.3),
    layers.Dense(256, activation='leaky_relu', kernel_initializer='he_normal'),
    layers.Dropout(0.3),
    layers.Dense(128, activation='relu'),
    layers.BatchNormalization(),
    layers.Dense(128, activation='swish', kernel_initializer='he_normal'),
    layers.LayerNormalization(),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(64, activation='swish', kernel_initializer='he_normal'),
    layers.Dropout(0.25),
    layers.Dense(32, activation='relu'),
    layers.BatchNormalization(),
    layers.Dense(32, activation='swish', kernel_initializer='he_normal'),
    layers.LayerNormalization(),
    layers.Dense(15, activation='linear')
])

rf_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
                    loss='mse', metrics=['mae'])

rf_model.fit(input_train_scaled, target_train_scaled, epochs=100, batch_size=8,
                validation_data=(input_test_scaled, target_test_scaled), verbose=0)

min_percentages = np.array([35, 25, 25, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0])
max_percentages = np.array([45, 35, 35, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0])

#Get all weightages for coke properties
ash_weightage = 1
vm_weightage = 1 #i think not needed 
m40_weightage = 1
m10_weightage = 1
csn_weightage = 1
cri_weightage = 1
ams_weightage = 1


ash_min = 15
ash_max = 17
vm_min = 0.5
vm_max = 1
m40_min = 90
m40_max = 93
m10_min = 5
m10_max = 7
csn_min = 65
csn_max = 70
cri_min = 22
cri_max = 26
ams_min = 53
ams_max = 56

#desired coke parameters
desired_ash = 15.5 #less
desired_VM = 0.75   #less
desired_m40 = 91.5   #higher
desired_m10 = 6   #less
desired_csr = 67.5   #higher
desired_cri = 24  #less
desired_ams = 54.5
desired_coke_parameters = [desired_ash,desired_VM,4,0.8,1.54,19,33,24,6,desired_m40,desired_m10,4,desired_csr,desired_cri,desired_ams]


def generate_combinations(index, current_combination, current_sum):
        target_sum = 100
        if index == len(min_percentages) - 1:
            remaining = target_sum - current_sum
            if min_percentages[index] <= remaining <= max_percentages[index]:
                yield current_combination + [remaining]
            return
        for value in range(min_percentages[index], max_percentages[index] + 1):
            if current_sum + value <= target_sum:
                yield from generate_combinations(index + 1, current_combination + [value], current_sum + value)

all_combinations = np.array(list(generate_combinations(0, [], 0)))

proces_para=[17, 10, 0.78, 1198,1255,1214 ,125,100,12.84,75,140,80,20,1050,5]
coal_costs = [0] * 33  # Initializing an array with 13 elements

# Example input: Replace these values with the actual costs
coal_costs = [18757,16700,16900,4500,19000,16209,16209,16209,16209,12000, 5200, 20160, 20160, 20160, 20160, 17000, 17000, 13440, 14200, 13440, 13440, 15000,9240,17000, 17000,17000, 17000,17000, 17000,17000, 17000,17000, 17000]


D_ = all_combinations
P_ = P
      
# Prepare tensors for new combinations
D_tensor = tf.constant(D_, dtype=tf.float32)
P_tensor = tf.constant(P_, dtype=tf.float32)

# Compute daily vectors for new combinations
daily_vectors = []
for i in range(D_tensor.shape[0]):
    row_vector = []
    for j in range(P_tensor.shape[1]):
        product_vector = tf.multiply(D_tensor[i], P_tensor[:, j])
        row_vector.append(product_vector)
    daily_vectors.append(tf.stack(row_vector))

daily_vectors_tensor = tf.stack(daily_vectors)
daily_vectors_flattened = daily_vectors_tensor.numpy().reshape(daily_vectors_tensor.shape[0], -1)

# Predict blended coal properties using pre-trained model
b1_scaled = input_scaler.transform(daily_vectors_flattened)
b1_scaled = b1_scaled.reshape(-1, get_coal_count, 15)
blend1 = modelq.predict(b1_scaled)
blended_coal_properties = output_scaler.inverse_transform(blend1)

blend1_with_process = blended_coal_properties + proces_para
blend1_scaled = input__scaler.transform(blend1_with_process)
coke = rf_model.predict(blend1_scaled)
predictions = output__scaler.inverse_transform(coke)

def filter_valid_and_invalid(predictions, combinations, blended_coal_properties):
    valid_indices = []
    invalid_indices = []
    for i, prediction in enumerate(predictions):
        # Check if all values are within the specified range
        if (
            ash_min <= prediction[0] <= ash_max and  # ASH
            vm_min <= prediction[1] <= vm_max and  # VM
            m40_min <= prediction[9] <= m40_max and  # M_40
            m10_min <= prediction[10] <= m10_max and  # M_10
            csn_min <= prediction[12] <= csn_max and  # CSR
            cri_min <= prediction[13] <= cri_max and # CRI
            ams_min <= prediction[14] <= ams_max
            
        ):
            valid_indices.append(i)
        else:
            invalid_indices.append(i)
    # Separate valid and invalid predictions, combinations, and blended coal properties
    valid_predictions = predictions[valid_indices]
    valid_combinations = combinations[valid_indices]
    valid_blended_coal_properties = [blended_coal_properties[i] for i in valid_indices]
    print("valid_blended_coal_properties","\n")
    print(valid_blended_coal_properties[0],"\n")
    print("blended_coal_properties","\n")
    print(blended_coal_properties[0],"\n")
    invalid_predictions = predictions[invalid_indices]
    invalid_combinations = combinations[invalid_indices]
    invalid_blended_coal_properties = [blended_coal_properties[i] for i in invalid_indices]

    return (
        valid_predictions,
        valid_combinations,
        valid_blended_coal_properties,
        invalid_predictions,
        invalid_combinations,
        invalid_blended_coal_properties,
    )

(valid_predictions, valid_combinations, valid_blended_coal_properties,
    invalid_predictions, invalid_combinations, invalid_blended_coal_properties) = \
    filter_valid_and_invalid(predictions, all_combinations, blended_coal_properties)

predictions = valid_predictions
all_combinations = valid_combinations
blended_coal_properties = valid_blended_coal_properties

# Initialize an array to store differences
differences = []
prediction = valid_predictions
all_combinations = valid_combinations

# Calculate differences
for prediction in predictions:
    diff = []
    
    # ASH (less is better)
    diff.append(((desired_ash - prediction[0]) / desired_ash) * ash_weightage)

    # VM (less is better)
    diff.append(((desired_VM - prediction[1]) / desired_VM) * vm_weightage)

    # M_40 (higher is better)
    diff.append(((prediction[9] - desired_m40) / desired_m40) * m40_weightage)

    # M_10 (less is better)
    diff.append(((desired_m10 - prediction[10]) / desired_m10) * m10_weightage)

    # CSR (higher is better)
    diff.append(((prediction[12] - desired_csr) / desired_csr) * csn_weightage)

    # CRI (less is better)
    diff.append(((desired_cri - prediction[13]) / desired_cri) * cri_weightage)
    
    # AMS (higher is better)
    diff.append(((prediction[14] - desired_csr) / desired_csr) * csn_weightage)

    differences.append(diff)

# Calculate total differences and store in an array
total_differences = [sum(diff) for diff in differences]
sorted_indices = np.argsort(total_differences)[::-1]

sorted_predictions = predictions[sorted_indices]
sorted_blends = all_combinations[sorted_indices]
sorted_diff = [differences[i] for i in sorted_indices]
sorted_blended_coal_properties = [blended_coal_properties[i] for i in sorted_indices]

total_costs = [sum(blend[i] * coal_costs[i] / 100 for i in range(32)) for blend in sorted_blends]

sorted_indices_by_cost = np.argsort(total_costs)

sorted_blend_cost = sorted_blends[sorted_indices_by_cost]
sorted_prediction_cost = sorted_predictions[sorted_indices_by_cost]
sorted_total_cost = np.array(total_costs)[sorted_indices_by_cost]
sorted_blended_coal_properties_cost = [sorted_blended_coal_properties[i] for i in sorted_indices_by_cost]
sorted_diff_cost = [sorted_diff[i] for i in sorted_indices_by_cost]

# Combine Cost and Performance
normalized_costs = (total_costs - np.min(total_costs)) / (np.max(total_costs) - np.min(total_costs))

normalized_differences = ((total_differences - np.min(total_differences)) /
                            (np.max(total_differences) - np.min(total_differences)))

cost_weight = 1
performance_weight = 1

combined_scores = (cost_weight * normalized_costs) + (performance_weight * normalized_differences)
best_combined_index = np.argmin(combined_scores)

# Helper for Cost Calculation
def calculate_cost(blend, coal_costs):
    return sum(blend[i] * coal_costs[i] / 100 for i in range(min(len(blend), len(coal_costs))))

np.set_printoptions(precision=3)


# 1. Best by performance (first in performance-sorted list)
blend_1 = sorted_blends[0]
blended_coal_1 = sorted_blended_coal_properties[0]
blend_1_properties = sorted_predictions[0]
blend_1_cost = calculate_cost(blend_1, coal_costs)

print(sorted_blends[50])
print(sorted_blends[32])
print(sorted_blends[20])

print(sorted_predictions[50])
print(sorted_predictions[32])
print(sorted_predictions[20])



# 2. Cheapest (first in cost-sorted list)
blend_2 = sorted_blend_cost[0]
blended_coal_2 = sorted_blended_coal_properties_cost[0]
blend_2_properties = sorted_prediction_cost[0]
blend_2_cost = sorted_total_cost[0]

# 3. Best combined (from original arrays, using best_combined_index)
blend_3 = all_combinations[best_combined_index]
blended_coal_3 = valid_blended_coal_properties[best_combined_index]
blend_3_properties = valid_predictions[best_combined_index]
blend_3_cost = calculate_cost(blend_3, coal_costs)

print("Blend 1 (First in Sorted Blends):")
print("Blend Values in %:", blend_1)
print("Properties:", blend_1_properties,"\n")
print("Blended Coal Parameters blend 1:", np.array2string(blended_coal_1, precision=3, suppress_small=True))
print("Cost:", blend_1_cost, "\n")

print("Blend 2 (Lowest Cost):")
print("Blend Values in %:", blend_2)
print("Properties:", blend_2_properties)
print("Blended Coal Parameters blend 2:", np.array2string(blended_coal_2, precision=3, suppress_small=True))
print("Cost:", blend_2_cost, "\n")

print("Blend 3 (Best Combined):")
print("Blend Values:", blend_3)
print("Properties:", blend_3_properties)
print("Blended Coal Parameters blend 3:", np.array2string(blended_coal_3, precision=3, suppress_small=True))
print("Cost:", blend_3_cost, "\n")


ValueError: Dimensions must be equal, but are 33 and 15 for '{{node compile_loss/mse/sub}} = Sub[T=DT_FLOAT](data_1, sequential_6_1/dense_76_1/BiasAdd)' with input shapes: [?,33], [?,15].

In [2]:
pip install itertools

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement itertools (from versions: none)
ERROR: No matching distribution found for itertools

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip
